In [1]:
import numpy as np
import cv2
import os
import PIL
import pathlib
from PIL import Image
import pyspark
from pyspark import SparkContext
import time

In [2]:
def returnName(url):
    flag = 0
    aux  = len(url)
    name = ''
    for i in np.arange(len(url)):
        if url[i] == 'I' :
            j = i
            while(aux > 0):
                name = name + url[j]
                j    = j + 1
                aux  = aux - 1
            return name
        aux = aux - 1

In [3]:
#Se define la ruta del directorio que contiene las imágenes.
data_dir = 'tilapias'
data_dir = pathlib.Path(data_dir)

In [4]:
list(data_dir.glob('*/*.JPG'))[:5]

[WindowsPath('tilapias/Hembras/IMG_2461.JPG'),
 WindowsPath('tilapias/Hembras/IMG_2462.JPG'),
 WindowsPath('tilapias/Hembras/IMG_2463.JPG'),
 WindowsPath('tilapias/Hembras/IMG_2464.JPG'),
 WindowsPath('tilapias/Machos/IMG_2446.JPG')]

In [5]:
#Se clasifican las imágenes en las clases "Hembras" y "Machos".
tilapias_images_dict = {
    'Hembras': list(data_dir.glob('Hembras/*')),
    'Machos' : list(data_dir.glob('Machos/*')),
}

In [6]:
#Se instancía SparkContext para poder realizar utilizar todos los métodos que nos provee
#y que se utilizarán en adelante.
sc   = SparkContext()
#El primer argumento es la colección de paths de la clase, y el segundo argumento es un
#parámetro que define el número de particiones que se crearán 
RDDM = sc.parallelize(tilapias_images_dict['Machos'],4)
RDDH = sc.parallelize(tilapias_images_dict['Hembras'],4)

In [7]:
RDDM.collect()

[WindowsPath('tilapias/Machos/IMG_2446.JPG'),
 WindowsPath('tilapias/Machos/IMG_2447.JPG'),
 WindowsPath('tilapias/Machos/IMG_2448.JPG'),
 WindowsPath('tilapias/Machos/IMG_2449.JPG')]

In [8]:
RDDH.collect()

[WindowsPath('tilapias/Hembras/IMG_2461.JPG'),
 WindowsPath('tilapias/Hembras/IMG_2462.JPG'),
 WindowsPath('tilapias/Hembras/IMG_2463.JPG'),
 WindowsPath('tilapias/Hembras/IMG_2464.JPG')]

In [10]:
RDDH.foreach(lambda x: print(x)) 

In [11]:
#NombreImagen -> Nombre del path de la imágen a editar.
#Factor       -> Grados que se desea rotar la imágen por iteración en un factor.
#Ruta         -> Ruta en la que se encuentran las imágenes.
def rotarImagenes(nombreImagen, factor, ruta):
    print("Entra a funcion de rotar")
    os.chdir(ruta)
    factorAux = factor
    #Para limpiar el path y dejar el nombre de la imágen.
    route = returnName(str(nombreImagen))
    #El factor auméntará en cada iteración hasta llegar a 360.
    while(factor < 360):
        newName = str(factor)
        picture = Image.open(route)
        newName = newName + route
        picture.rotate(factor).save(newName)
        factor  = factor +factorAux

In [12]:
ls

 El volumen de la unidad C es Windows
 El n�mero de serie del volumen es: 2296-934F

 Directorio de c:\Users\austi\Documents\OCTAVO SEMESTRE\Sistemas de computo paralelo y distribuido\SPD\Spark

25/06/2022  07:52 p. m.    <DIR>          .
25/06/2022  07:52 p. m.    <DIR>          ..
25/06/2022  07:52 p. m.    <DIR>          .ipynb_checkpoints
26/06/2022  12:50 a. m.             8,503 imagesGenerator.ipynb
25/06/2022  07:52 p. m.    <DIR>          tilapias
               1 archivos          8,503 bytes
               4 dirs  229,140,205,568 bytes libres


In [13]:
registroTiempos = []
start = time.time()
route = 'tilapias/Hembras'
RDDH.foreach(lambda x: rotarImagenes(nombreImagen=x, factor=1, ruta=route)) 

In [14]:

route = 'tilapias/Machos'
RDDM.foreach(lambda x: rotarImagenes(nombreImagen=x, factor=1, ruta=route)) 
stop = time.time()
registroTiempos.append((stop-start))
print(registroTiempos[0])

106.40552806854248
